In [1]:
import mysql.connector as sql
import os, warnings
import pandas as pd
import numpy as np
from dotenv import load_dotenv
load_dotenv()
warnings.simplefilter(action='ignore', category=FutureWarning)

conn = sql.connect(
    host = os.getenv("MYSQL_HOST"),
    user = os.getenv("MYSQL_USER"),
    password = os.getenv("MYSQL_PASSWORD"),
    database = 'DataAnalysis',
    port = '3306')
query = ''' SELECT ZillowListings.Id, ZillowListings.Beds, ZillowListings.Baths, ZillowListings.CityName,
ZillowListingsWalkscore.WalkScore, ZillowListingsWalkscore.TransitScore,
ZillowListings.ListingType, AVG(ZillowListingsAssociations.Price) AS Price,
COUNT(ZillowListingsAirbnb.Id) AS Airbnbs,
CASE
    WHEN AVG(ZillowListingsAssociations.Price) > Overall_Avg.AvgPrice THEN 'A'
    WHEN AVG(ZillowListingsAssociations.Price) < Overall_Avg.AvgPrice THEN 'B'
    ELSE 'E'
END AS value_comparison
FROM ZillowListingsAssociations
INNER JOIN ZillowListings ON 
ZillowListingsAssociations.Id = ZillowListings.Id
INNER JOIN ZillowListingsWalkscore ON 
ZillowListings.Id = ZillowListingsWalkscore.Id
INNER JOIN ZillowListingsColleges ON
ZillowListings.Id = ZillowListingsColleges.Id
INNER JOIN ZillowListingsAirbnb ON
ZillowListings.Id = ZillowListingsAirbnb.Id
INNER JOIN (SELECT AVG(ZillowListingsAssociations.Price) AS AvgPrice FROM ZillowListingsAssociations) AS Overall_Avg
ON 1=1
WHERE Price > 10000
GROUP BY ZillowListings.Id, ZillowListings.Beds, ZillowListings.Baths, ZillowListings.CityName,
ZillowListings.ListingType, Overall_Avg.AvgPrice '''

cursor = conn.cursor(buffered=True , dictionary=True)

In [2]:
df = pd.read_sql(query, conn)
conn.close()

C:\Users\Kyle\AppData\Local\Temp\ipykernel_15432\2049290334.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [3]:
df.head()

,Id,Beds,Baths,CityName,WalkScore,TransitScore,ListingType,Price,Colleges,Airbnbs,value_comparison
0,2054418145,2.0,2.0,Ottawa,32.0,49.0,CONDO,3.500000e+05,42,42,B
1,2054424380,4.0,3.0,Ottawa,15.0,38.0,SINGLE_FAMILY,6.970429e+05,224,224,B
2,2054432735,4.0,3.0,Ottawa,48.0,37.0,SINGLE_FAMILY,9.980000e+05,1620,1620,B
3,2054481385,3.0,2.0,Ottawa,23.0,38.0,SINGLE_FAMILY,5.399000e+05,288,288,B
4,2054526113,3.0,2.0,Ottawa,58.0,77.0,SINGLE_FAMILY,1.173000e+06,70,70,A


In [16]:
df["value_comparison"] = (df["value_comparison"] == "AA").astype('int')
df.head(100)

,Id,Beds,Baths,CityName,WalkScore,TransitScore,ListingType,Price,value_comparison
0,2054238156,3.0,2.0,Pickering,62.0,58.0,CONDO,549999.0,1
1,2054238517,2.0,1.0,Arnprior,80.0,NaN,CONDO,265000.0,0
2,2054238804,2.0,3.0,Ottawa,69.0,38.0,CONDO,429900.0,0
3,2054238816,2.0,2.0,Guelph,69.0,48.0,CONDO,699900.0,1
4,2054239098,1.0,1.0,Toronto,97.0,76.0,CONDO,749900.0,1
...,...,...,...,...,...,...,...,...,...
95,2054248338,1.0,1.0,Richmond Hill,74.0,62.0,CONDO,525000.0,0
96,2054248376,1.0,2.0,Vaughan,60.0,49.0,CONDO,579900.0,1
97,2054248647,1.0,1.0,Penetanguishene,15.0,NaN,CONDO,329900.0,0
98,2054248695,2.0,2.0,Burlington,77.0,34.0,CONDO,675000.0,1
